In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
import torchvision.transforms as T
import torchvision.datasets as dset
from torch.utils.data import DataLoader
import random
import numpy as np

In [ ]:
from transformers import ViTForImageClassification, ViTImageProcessor

In [ ]:
# Hyper Parameter
NUM_EPOCHS = 10
BATCH_SIZE = 16
LEARNING_RATE = 1e-4

PRINT_EVERY = 100

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
FOLDERPATH = 'Colab\ Notebooks/Poster'

In [ ]:
%cd drive/MyDrive/$FOLDERPATH

/content/drive/MyDrive/Colab Notebooks/Poster


In [ ]:
%ls

 Dataset/                               module_ResNet50.ipynb
 deepfake0.ipynb                        module_VIiT_0908.ipynb
 Densenet0910.ipynb                     resnet_with_avgpool.ipynb
 milestone@                            'Saliency_map_and_Confusion_matrix(unfinish).ipynb'
 model_0907_weights.pth                 test/
 model_0908_weights.pth                 train/
 model_0910E_weights.pth                val/
 model_0910_weights.pth                 vit_0910_10_weights.pth
 model_D0908_weights.pth                vit_0910_20_weights.pth
 model_Densenet0913_weights.pth         vit_0912_30_weights.pth
 model_EfficientNetb70910_weights.pth


In [ ]:
# Check device
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(f'Device: {device}')

Device: cuda


In [ ]:
# Fix the random seed to compare the result
# Resource：https://yanwei-liu.medium.com/pytorch-reproducibility-db8458111b75

def set_seed(seed=42, loader=None):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.benchmark = False
  torch.backends.cudnn.deterministic = True
  try:
    loader.sampler.generator.manual_seed(seed)
  except AttributeError:
    pass

set_seed()

In [ ]:
# Data pre-processing
transform = T.Compose([
    T.Resize((300,300)),
    T.RandomCrop((224,224)),
    T.ToTensor()
])
transform_test = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])

In [ ]:
# Load Data
# 0: portrait  1: midjourney  2: SD  3: bing
train_data = dset.ImageFolder('Dataset/train', transform=transform)
val_data = dset.ImageFolder('Dataset/val', transform=transform)
test_data = dset.ImageFolder('Dataset/test', transform=transform_test)

In [ ]:
# Check data dimension
print(train_data[0][0].shape)
print(val_data[0][1])

torch.Size([3, 224, 224])
0


In [ ]:
NUM_TRAIN = len(train_data)
NUM_VAL = len(val_data)
NUM_TEST = len(test_data)
print('Number of training:', NUM_TRAIN)
print('Number of validation:', NUM_VAL)
print('Numver of test:', NUM_TEST)

Number of training: 1400
Number of validation: 140
Numver of test: 140


In [ ]:
# Set seed for DataLoader
def seed_worker(worker_id):
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

In [ ]:
# Create Mini-Batch
mini_trains = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, worker_init_fn=seed_worker, generator=g)
mini_vals = DataLoader(val_data, batch_size=BATCH_SIZE, worker_init_fn=seed_worker, generator=g)
mini_tests = DataLoader(test_data, batch_size=BATCH_SIZE)

In [ ]:
# Build model
vit = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
num_features = vit.config.hidden_size
vit.classifier = nn.Linear(num_features, 4)
model = vit.to(device)

In [ ]:
# Define loss function & optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

NameError: name 'model' is not defined

In [ ]:
# Training procedure
def train(mini_trains, model, NUM_EPOCHES, device):
  for epoch in range(NUM_EPOCHES):
    for count, (x, y) in enumerate(mini_trains):
      # Turn on training mode
      model.train()
      # Move data to device
      x = x.to(device)
      y = y.to(device)
      # ForwardProp
      scores = model(x)
      # Extract logits from the output obeject
      scores = scores.logits
      # Calculate loss
      loss = loss_function(scores, y)
      if count % PRINT_EVERY == 0:
        print(f'[Epoch {epoch+1}] Training loss', loss.item())
        validation(mini_vals, epoch, model, device)
      # Clear the previous gradients
      optimizer.zero_grad()
      # Get the gradients
      loss.backward()
      # Update the weights
      optimizer.step()
    print('-'*50)

In [ ]:
# Validation Procedure
def validation(mini_vals, epoch, model, device):
  # Turn on val mode
  model.eval()
  with torch.no_grad():
    acc_acount = 0
    for x, y in mini_vals:
      # move data to device
      x = x.to(device)
      y = y.to(device)
      # Forward Prop
      scores = model(x)
      # Extract logits from the output obeject
      scores = scores.logits
      # Calculate accuracy
      predictions = scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_acount += acc
    print(f'[Epoch {epoch+1}] Accuracy:', acc_acount/NUM_VAL)

In [ ]:
# Training
train(mini_trains, model, NUM_EPOCHS, device)

[Epoch 1] Training loss 1.3277947902679443
[Epoch 1] Accuracy: 0.21428571428571427
--------------------------------------------------
[Epoch 2] Training loss 0.5594815015792847
[Epoch 2] Accuracy: 0.6714285714285714
--------------------------------------------------
[Epoch 3] Training loss 0.4221544861793518
[Epoch 3] Accuracy: 0.7785714285714286
--------------------------------------------------
[Epoch 4] Training loss 0.08790064603090286
[Epoch 4] Accuracy: 0.7785714285714286
--------------------------------------------------
[Epoch 5] Training loss 0.011679100804030895
[Epoch 5] Accuracy: 0.8071428571428572
--------------------------------------------------
[Epoch 6] Training loss 0.042338237166404724
[Epoch 6] Accuracy: 0.7571428571428571
--------------------------------------------------
[Epoch 7] Training loss 0.033792611211538315
[Epoch 7] Accuracy: 0.7857142857142857
--------------------------------------------------
[Epoch 8] Training loss 0.06251560151576996
[Epoch 8] Accurac

In [ ]:
# Test score procedure
def test(mini_tests, model, device):
  # Use val mode to test
  model.eval()
  with torch.no_grad():
    acc_count = 0
    for x, y in mini_tests:
      # move data to device
      x = x.to(device)
      y = y.to(device)
      # Last Forward Prop
      score_test = model(x)
      # Extract logits from the output obeject
      score_test = score_test.logits
      # Calculate Accuracy
      predictions = score_test.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'[Final] Test Accuracy:', acc_count/NUM_TEST)

In [ ]:
test(mini_tests, model, device)

[Final] Test Accuracy: 0.7214285714285714


In [ ]:
# save trained weights
torch.save(vit.state_dict(), 'vit_0910_10_weights.pth')

In [ ]:
# Load model and trained weights
PATH = 'vit_0910_10_weights.pth'

vit = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
# Modify the classifier head to fit your 4-class classification problem
num_features = vit.config.hidden_size
vit.classifier = nn.Linear(num_features, 4)
vit.load_state_dict(torch.load(PATH))
model = vit.to(device)

<ipython-input-44-7591ee5599aa>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vit.load_state_dict(torch.load(PATH))


In [ ]:
transform_test = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])

In [ ]:
# Load test data
new_test_data = dset.ImageFolder('milestone', transform=transform_test)
NUM_NEW_TEST = len(new_test_data)
print('Number of new test:', NUM_NEW_TEST)
mini_tests = DataLoader(new_test_data, batch_size=1)

Number of new test: 400


In [ ]:
new_test_data[0][0].shape

torch.Size([3, 224, 224])

In [ ]:
# Test score procedure
def milestone_test(mini_tests, model, device):
    # Use eval mode to test
    model.eval()
    valid_classes = [0, 2]  # Classes Filter
    with torch.no_grad():
        acc_count = 0
        total_samples = 0
        for x, y in mini_tests:
            # move data to device
            x = x.to(device)
            y = y.to(device)

            # Last Forward Prop
            score = model(x)

            # Extract logits from the output obeject
            score = score.logits

            # Calculate predictions
            predictions = score.max(1)[1]

            # Filter out predictions that are not in valid_classes
            filtered_predictions = [p.item() for p in predictions if p.item() in valid_classes]
            filtered_labels = [label.item() for label in y if label.item() in valid_classes]

            # Calculate Accuracy
            correct_predictions = sum([pred == label for pred, label in zip(filtered_predictions, filtered_labels)])
            acc_count += correct_predictions
            total_samples += len(filtered_labels)  # Update with the number of valid samples

        print(f'[Final] Test Accuracy:', acc_count / total_samples if total_samples > 0 else 0)

In [ ]:
milestone_test(mini_tests, model, device)

[Final] Test Accuracy: 0.755


# 10 more epochs

In [ ]:
ADD_EPOCHS = 10

In [ ]:
# Load model and trained weights
PATH = 'vit_0912_30_weights.pth'

vit = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
# Modify the classifier head to fit your 4-class classification problem
num_features = vit.config.hidden_size
vit.classifier = nn.Linear(num_features, 4)
vit.load_state_dict(torch.load(PATH))
model_2 = vit.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

<ipython-input-17-cfcb3a43a2c3>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vit.load_state_dict(torch.load(PATH))


In [ ]:
# Define loss function & optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_2.parameters(), lr=LEARNING_RATE)

In [ ]:
# Training procedure
def train(mini_trains, model, NUM_EPOCHES, device):
  for epoch in range(NUM_EPOCHES):
    for count, (x, y) in enumerate(mini_trains):
      # Turn on training mode
      model.train()
      # Move data to device
      x = x.to(device)
      y = y.to(device)
      # ForwardProp
      scores = model(x)
      # Extract logits from the output obeject
      scores = scores.logits
      # Calculate loss
      loss = loss_function(scores, y)
      if count % PRINT_EVERY == 0:
        print(f'[Epoch {epoch+1}] Training loss', loss.item())
        validation(mini_vals, epoch, model, device)
      # Clear the previous gradients
      optimizer.zero_grad()
      # Get the gradients
      loss.backward()
      # Update the weights
      optimizer.step()
    print('-'*50)

In [ ]:
# Validation Procedure
def validation(mini_vals, epoch, model, device):
  # Turn on val mode
  model.eval()
  with torch.no_grad():
    acc_acount = 0
    for x, y in mini_vals:
      # move data to device
      x = x.to(device)
      y = y.to(device)
      # Forward Prop
      scores = model(x)
      # Extract logits from the output obeject
      scores = scores.logits
      # Calculate accuracy
      predictions = scores.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_acount += acc
    print(f'[Epoch {epoch+1}] Accuracy:', acc_acount/NUM_VAL)

In [ ]:
# Training
train(mini_trains, model_2, ADD_EPOCHS, device)

[Epoch 1] Training loss 0.002948096487671137
[Epoch 1] Accuracy: 0.7571428571428571
--------------------------------------------------
[Epoch 2] Training loss 0.05931933969259262
[Epoch 2] Accuracy: 0.6928571428571428
--------------------------------------------------
[Epoch 3] Training loss 0.0003769036557059735
[Epoch 3] Accuracy: 0.7
--------------------------------------------------
[Epoch 4] Training loss 0.0013694466324523091
[Epoch 4] Accuracy: 0.7428571428571429
--------------------------------------------------
[Epoch 5] Training loss 0.02244730107486248
[Epoch 5] Accuracy: 0.7357142857142858
--------------------------------------------------
[Epoch 6] Training loss 9.733380284160376e-05
[Epoch 6] Accuracy: 0.6428571428571429
--------------------------------------------------
[Epoch 7] Training loss 0.0002661596518009901
[Epoch 7] Accuracy: 0.7642857142857142
--------------------------------------------------
[Epoch 8] Training loss 0.0010089417919516563
[Epoch 8] Accuracy: 0.

In [ ]:
# Test score procedure
def test(mini_tests, model, device):
  # Use val mode to test
  model.eval()
  with torch.no_grad():
    acc = 0
    acc_count = 0
    for x, y in mini_tests:
      # move data to device
      x = x.to(device)
      y = y.to(device)
      # Last Forward Prop
      score_test = model(x)
      # Extract logits from the output obeject
      score_test = score_test.logits
      # Calculate Accuracy
      predictions = score_test.max(1)[1]
      acc = predictions.eq(y).sum().item()
      acc_count += acc
    print(f'[Final] Test Accuracy:', acc_count/NUM_TEST)

In [ ]:
test(mini_tests, model_2, device)

[Final] Test Accuracy: 0.7785714285714286


In [ ]:
# save trained weights
torch.save(vit.state_dict(), 'vit_0913_40_weights.pth')

In [ ]:
transform_test = T.Compose([
    T.Resize((224,224)),
    T.ToTensor()
])

In [ ]:
# Load test data
new_test_data = dset.ImageFolder('milestone', transform=transform_test)
NUM_NEW_TEST = len(new_test_data)
print('Number of new test:', NUM_NEW_TEST)
mini_tests = DataLoader(new_test_data, batch_size=1)

Number of new test: 400


In [ ]:
# Test score procedure
def milestone_test(mini_tests, model, device):
    # Use eval mode to test
    model.eval()
    valid_classes = [0, 2]  # Classes Filter
    with torch.no_grad():
        acc_count = 0
        total_samples = 0
        for x, y in mini_tests:
            # move data to device
            x = x.to(device)
            y = y.to(device)

            # Last Forward Prop
            score = model(x)

            # Extract logits from the output obeject
            score = score.logits

            # Calculate predictions
            predictions = score.max(1)[1]

            # Filter out predictions that are not in valid_classes
            filtered_predictions = [p.item() for p in predictions if p.item() in valid_classes]
            filtered_labels = [label.item() for label in y if label.item() in valid_classes]

            # Calculate Accuracy
            correct_predictions = sum([pred == label for pred, label in zip(filtered_predictions, filtered_labels)])
            acc_count += correct_predictions
            total_samples += len(filtered_labels)  # Update with the number of valid samples

        print(f'[Final] Test Accuracy:', acc_count / total_samples if total_samples > 0 else 0)

In [ ]:
milestone_test(mini_tests, model_2, device)

[Final] Test Accuracy: 0.67
